In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
#import glove
#from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

#from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
new_notes = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/ner_df_final.p") # med7
w2vec = Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/embeddings/word2vec.model")
fasttext = FastText.load("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/embeddings/fasttext.model")
#word2vec = gensim.models.KeyedVectors.load('word2vec.model')

In [ ]:
null_index_list = []
for i in new_notes.itertuples():
    
    if len(i.ner) == 0:
    #if not i.ner:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)
print(len(new_notes))

119519


In [ ]:
med7_ner_data = {}

for ii in new_notes.itertuples():
    
    p_id = ii.SUBJECT_ID
    ind = ii.Index
    
    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []
            
    unique = set()
    new_temp = []
    
    for j in new_ner:
        for k in j:
            
            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp
print(len(med7_ner_data))

21837


In [ ]:
pd.to_pickle(med7_ner_data, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner_word_dict.pkl")

In [ ]:
def mean(a):
    return sum(a) / len(a)

In [ ]:
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(w2vec[i[0]])
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    #t = np.asarray(map(mean, zip(*avg)))
                    t = np.asarray(list(map(mean, zip(*avg))))
                    patient_temp.append(t)
        if len(patient_temp) == 0: continue
        new_word2vec[k] = patient_temp
    print(len(new_word2vec))
    #############################################################################
    print("fasttext starting..")
        
    new_fasttextvec = {}

    for k,v in data.items():

        patient_temp = []

        for i in v:
            try:
                patient_temp.append(fasttext[i[0]])
            except:
                pass
        if len(patient_temp) == 0: continue
        new_fasttextvec[k] = patient_temp
    print(len(new_fasttextvec))
    #############################################################################    

    print("combined starting..")
    new_concatvec = {}

    for k,v in data.items():
        patient_temp = []
    #     if k != 6: continue
        for i in v:
            w2vec_temp = []
            fasttemp = [] # nitin added 
            try:
                w2vec_temp = w2vec[i[0]]
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: 
                        #w2vec_temp = [0] * 100
                        w2vec_temp = [0] * 200
                    else:
                        avg = np.asarray(avg)
                        w2vec_temp = np.asarray(list(map(mean, zip(*avg)))) # nitin added list after map for fixing the return of map as hash 
                else:
                    #w2vec_temp = [0] * 100
                    w2vec_temp = [0] * 200
            #print('value for i',i)    # nitin added the try pass to eliminate the error if i[[0]] doesn't exist
            try:
                fasttemp = fasttext[i[0]]
            except:
                #fasttemp = [0] * 100
                fasttemp = [0] * 200
            #print('fasttemp {0}, w2vec_temp{1},'.format(fasttemp,w2vec_temp))
            appended = np.append(fasttemp, w2vec_temp, 0)
            patient_temp.append(appended)
        if len(patient_temp) == 0: continue
        new_concatvec[k] = patient_temp



    #print(len(new_word2vec), len(new_fasttextvec))
    print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))
    #new_word2vec
    #new_fasttextvec
    pd.to_pickle(new_word2vec, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner5_word2vec_dict.pkl")
    pd.to_pickle(new_fasttextvec, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner5_fasttext_dict.pkl")
    pd.to_pickle(new_concatvec, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner5_combined_dict.pkl")

w2vec starting..
21731
fasttext starting..
21809
combined starting..
21731 21809 21837


In [ ]:
#print("new_word2vec", new_word2vec[0])
new_word2vec_dict = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner5_word2vec_dict.pkl")
new_fasttext_dict = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner5_fasttext_dict.pkl")
new_combined_dict = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner5_combined_dict.pkl")
diff = set(new_fasttext_dict.keys()).difference(set(new_word2vec_dict))
for i in diff:
    del new_fasttext_dict[i]
    del new_combined_dict[i]
print (len(new_word2vec_dict), len(new_fasttext_dict), len(new_combined_dict))
#print (len(new_word2vec_dict), len(new_fasttext_dict))


pd.to_pickle(new_word2vec_dict, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+"new_ner5_word2vec_limited_dict.pkl")
pd.to_pickle(new_fasttext_dict, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+"new_ner5_fasttext_limited_dict.pkl")
pd.to_pickle(new_fasttext_dict, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+"new_ner5_combined_limited_dict.pkl")
#pd.to_pickle(new_combined_dict, "data/"+"new_ner"+"_combined_limited_dict.pkl")

21731 21731 21759


In [ ]:
lvl2_train =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/lvl2_imputer_train.pkl")
lvl2_dev =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/lvl2_imputer_dev.pkl")
lvl2_test =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/lvl2_imputer_test.pkl")

Ys =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/Ys.pkl")
Ys_train =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/Ys_train.pkl")
Ys_dev =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/Ys_dev.pkl")
Ys_test =  pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/Ys_test.pkl")

In [ ]:
all_train_ids = set()
for i in Ys_train.itertuples():
    all_train_ids.add( i.Index[0] )
    
all_dev_ids = set()
for i in Ys_dev.itertuples():
    all_dev_ids.add( i.Index[0] )
    
all_test_ids = set()
for i in Ys_test.itertuples():
    all_test_ids.add( i.Index[0] )

In [ ]:
print (sum(Ys_train.mort_icu.values)*1.0 / len(Ys_train.mort_icu.values))
print (sum(Ys_dev.mort_icu.values)*1.0 / len(Ys_dev.mort_icu.values))
print (sum(Ys_test.mort_icu.values)*1.0 / len(Ys_test.mort_icu.values))
print ("====")
print (sum(Ys_train.mort_hosp.values)*1.0 / len(Ys_train.mort_hosp.values))
print (sum(Ys_dev.mort_hosp.values)*1.0 / len(Ys_dev.mort_hosp.values))
print (sum(Ys_test.mort_hosp.values)*1.0 / len(Ys_test.mort_hosp.values))
print ("====")
print (sum(Ys_train.los_3.values)*1.0 / len(Ys_train.los_3.values))
print (sum(Ys_dev.los_3.values)*1.0 / len(Ys_dev.los_3.values))
print (sum(Ys_test.los_3.values)*1.0 / len(Ys_test.los_3.values))
print ("====")
print (sum(Ys_train.los_7.values)*1.0 / len(Ys_train.los_7.values))
print (sum(Ys_dev.los_7.values)*1.0 / len(Ys_dev.los_7.values))
print (sum(Ys_test.los_7.values)*1.0 / len(Ys_test.los_7.values))

0.07159904534606205
0.06725146198830409
0.07432150313152401
====
0.10680190930787589
0.09649122807017543
0.10855949895615867
====
0.43323389021479713
0.42105263157894735
0.4246346555323591
====
0.07696897374701671
0.07268170426065163
0.07954070981210856


In [ ]:
new_word2vec_dict = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner4_word2vec_dict.pkl")
#new_word2vec_dict = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner_word_dict.pkl")
new_keys = set(new_word2vec_dict.keys())
new_train_ids = sorted(all_train_ids.intersection(new_keys))
new_dev_ids = sorted(all_dev_ids.intersection(new_keys))
new_test_ids = sorted(all_test_ids.intersection(new_keys))

In [ ]:
#new_train_ids = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_train_ids.pkl")
#new_dev_ids = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_dev_ids.pkl")
#new_test_ids = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_test_ids.pkl")

In [ ]:
print("new_train_ids = ", new_train_ids)
print("new_dev_ids = ", new_dev_ids)
print("new_test_ids = ", new_test_ids)

new_train_ids =  [3, 9, 12, 13, 17, 19, 21, 25, 30, 31, 35, 41, 45, 52, 55, 56, 61, 62, 64, 65, 68, 71, 78, 81, 83, 85, 88, 97, 99, 100, 101, 103, 105, 106, 114, 115, 123, 124, 127, 129, 130, 133, 134, 135, 137, 140, 141, 143, 144, 146, 147, 149, 152, 160, 163, 169, 170, 171, 173, 174, 175, 177, 178, 186, 191, 195, 201, 202, 205, 208, 209, 211, 212, 214, 222, 225, 226, 228, 234, 238, 242, 245, 249, 251, 253, 255, 256, 261, 262, 265, 267, 268, 269, 270, 271, 272, 273, 274, 275, 279, 281, 287, 290, 298, 301, 302, 305, 306, 307, 309, 310, 313, 314, 315, 318, 319, 321, 323, 326, 329, 330, 335, 344, 345, 346, 347, 350, 351, 353, 354, 356, 360, 364, 366, 367, 370, 371, 379, 383, 389, 391, 394, 395, 396, 397, 400, 402, 406, 407, 409, 417, 418, 420, 421, 422, 424, 429, 434, 436, 437, 438, 439, 440, 445, 448, 452, 453, 458, 462, 464, 466, 468, 471, 472, 477, 481, 482, 485, 487, 489, 491, 492, 498, 503, 507, 510, 518, 523, 533, 536, 538, 539, 540, 543, 545, 554, 557, 558, 559, 561, 564, 569, 577

In [ ]:
data_ids = [(new_train_ids, new_dev_ids, new_test_ids)]
data_names = ["new"]

for i, (tr, de, te) in zip(data_names, data_ids):
    
    y_train = Ys_train.loc[tr]
    y_dev = Ys_dev.loc[de]
    y_test = Ys_test.loc[te]

    sub_train = lvl2_train.loc[tr]
    sub_train = sub_train.loc[:, pd.IndexSlice[:, 'mean']]

    sub_dev = lvl2_dev.loc[de]
    sub_dev = sub_dev.loc[:, pd.IndexSlice[:, 'mean']]

    sub_test = lvl2_test.loc[te]
    sub_test = sub_test.loc[:, pd.IndexSlice[:, 'mean']]
    #print(sub_train)
    #sub_train = sub_train.as_matrix()
    #sub_dev = sub_dev.as_matrix()
    #sub_test = sub_test.as_matrix()

    sub_train = sub_train.to_numpy()
    sub_dev = sub_dev.to_numpy()
    sub_test = sub_test.to_numpy()

    # reshape the data for timeseries prediction
    x_train_lstm = sub_train.reshape(int(sub_train.shape[0] / 24), 24, 104)
    x_dev_lstm = sub_dev.reshape(int(sub_dev.shape[0] / 24), 24, 104)
    x_test_lstm = sub_test.reshape(int(sub_test.shape[0] / 24), 24, 104)

    
    pd.to_pickle(x_train_lstm, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+i+"_x_train.pkl")
    pd.to_pickle(x_dev_lstm, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+i+"_x_dev.pkl")
    pd.to_pickle(x_test_lstm, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+i+"_x_test.pkl")
    
    pd.to_pickle(y_train, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+i+"_y_train.pkl")
    pd.to_pickle(y_dev, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+i+"_y_dev.pkl")
    pd.to_pickle(y_test, "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+i+"_y_test.pkl")

In [ ]:
#import statistics
#from statistics import mean
def create_dataset(dict_of_ner):
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
        temp_data.append(np.mean(temp, axis = 0)) 
    return np.asarray(temp_data)
#def create_dataset(dict_of_ner):
#    temp_data = []
#    for k, v in sorted(dict_of_ner.items()):
#        temp = []
#        for embed in v:
            #print(type(v))
            #ndArray= np.array(embed, dtype=float)
#            temp.append(ndArray)
            #temp.extend(embed)
            #print("ndArray[0] ", type(ndArray[0]))
            ##rint("ndArray ", type(ndArray))
        #print(temp.shape)
        #print(len(temp))
        #temp1 = np.mean(temp, axis=0)
        #temp = np.asarray(temp)
        #temp1 = list(map(mean, zip(*temp)))
        #temp = np.asarray(temp)
        #print(temp)
        #print(temp.shape)
        #temp1 = np.mean(temp)
        #temp1 = np.mean(temp, axis=0)
        #temp_data.append(temp1)
        #temp1 = np.array(list(map(float, temp)))
        #temp1 = np.asarray(list(map(float, temp)))
        #temp1 = np.sum(temp)
        #temp1 = sum(float(t) for t in temp)
        #temp1 = map(mean(temp))
        #temp = np.asarray(temp)
        #length = max(map(len, temp))
        #temp = np.array([xi+[None]*(length-len(xi)) for xi in temp])
        #temp = np.array([np.array(xi) for xi in temp])
        #print(temp.shape)
        #print(temp)
        #temp1 = np.asarray(list(map(mean, zip(*temp))))
        #temp1 = np.stack(temp, axis=0)
        #temp2 = np.mean(temp1)
        #print(temp2)
        #lists = temp
        #flat_list = [item for sublist in lists for item in sublist]
        #print(np.mean(flat_list))
#        print(type(temp[0]))
#        temp1 = np.mean(temp)
        #print(temp1)
#        temp_data.append(temp1)
 
#    return np.asarray(temp_data)

def make_prediction_multi_avg(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_multi_avg(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1
    
    result_path = "/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/results/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-avg-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def avg_ner_model(layer_name, number_of_unit, embedding_name):

    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_avg = Input(shape=(input_dimension, ), name = "avg")        
#     x_1 = Dense(256, activation='relu')(input_avg)
#     x_1 = Dropout(0.3)(x_1)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    x = keras.layers.Concatenate()([x, input_avg])

    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    
    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    #opt = Adam(lr=0.001, decay = 0.01)
    model = Model(inputs=[sequence_input, input_avg], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    
    return model

In [ ]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_y_test.pkl")

#ner_word2vec = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
#ner_fasttext = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
#ner_concat = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_ner_combined_limited_dict.pkl")
ner_word2vec = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner4_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner4_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/new_ner4_combined_limited_dict.pkl")

new_keys = set(new_word2vec.keys())
train_ids = sorted(all_train_ids.intersection(new_keys))
dev_ids = sorted(all_dev_ids.intersection(new_keys))
test_ids = sorted(all_test_ids.intersection(new_keys))
#train_ids = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_train_ids.pkl")
#dev_ids = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_dev_ids.pkl")
#test_ids = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ConvolutionMedicalNer-master/data/"+type_of_ner+"_test_ids.pkl")

In [ ]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

In [ ]:
#embedding_types = ['concat']
#embedding_dict = [ner_concat]
#embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]
embedding_types = ['word2vec', 'fasttext']
#embedding_dict = [ner_word2vec, ner_fasttext]
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']


num_epoch = 100
#num_epoch = 2
model_patience = 5
monitor_criteria = 'val_loss'
batch_size = 64
iter_num = 11
#unit_sizes = [128, 256]
unit_sizes = [256]

#layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print ("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print ("Hidden unit: ", each_unit_size)

        for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
            print ("Embedding: ", embed_name)
            print("=============================")

            temp_train_ner = dict((k, ner_word2vec[k]) for k in train_ids)
            temp_dev_ner = dict((k, ner_word2vec[k]) for k in dev_ids)
            temp_test_ner = dict((k, ner_word2vec[k]) for k in test_ids)

            x_train_ner = create_dataset(temp_train_ner)
            x_dev_ner = create_dataset(temp_dev_ner)
            x_test_ner = create_dataset(temp_test_ner)


            for iteration in range(1, iter_num):
                print ("Iteration number: ", iteration)

                for each_problem in target_problems:
                    print ("Problem type: ", each_problem)
                    print ("__________________")

                    early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                    best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                    checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                        save_best_only=True, mode='min', period=1)


                    callbacks = [early_stopping_monitor, checkpoint]

                    model = avg_ner_model(each_layer, each_unit_size, embed_name)
                    
                    model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                              validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, 
                              batch_size=batch_size )

                    model.load_weights(best_model_name)

                    probs, predictions = make_prediction_multi_avg(model, [x_test_lstm, x_test_ner])
                    
                    save_scores_multi_avg(predictions, probs, y_test[each_problem], 
                                embed_name, each_problem, iteration, each_unit_size, 
                                each_layer, type_of_ner)
                    
                    reset_keras(model)
                    #del model
                    clear_session()
                    gc.collect()

Layer:  GRU
Hidden unit:  256
Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2885 - acc: 0.9030
Epoch 1: val_loss improved from inf to 0.24204, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 26s 101ms/step - loss: 0.2885 - acc: 0.9030 - val_loss: 0.2420 - val_acc: 0.9168
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.2407 - acc: 0.9141
Epoch 2: val_loss improved from 0.24204 to 0.23059, saving model to avg-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 26s 108ms/step - loss: 0.2407 - acc: 0.9141 - val_loss: 0.2306 - val_acc: 0.9168
Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.2212 - acc: 0.9207
Epoch 3: val_loss did not improve from 0.23059
238/238 [==============================] - 24s 102ms/step - loss: 0.2212 - acc: 0.9207 - val_loss: 0.24